In [31]:
import pandas as pd

def mandatumkalkulacio(
    csv_path,
    orszagos_eredmenyek: dict,
    reszvetel_szazalek: float = 70.0,
    kulhoni_szavazatok: dict = None,
    fix_mandatumok: dict = None,
    taktikai_atszavazas: dict = None,
    taktikai_atszavazas_korzet: dict = None,
    output_path: str = "mandatum_kalkulacio_eredmeny.csv"
):
    df = pd.read_csv(csv_path, sep=';', encoding='utf-8-sig')
    df.drop(columns=['Index'], inplace=True, errors='ignore')  # Hibakezelés, ha nincs Index oszlop

    parties = [
        "Fidesz (%)",
        "Tisza (%)",
        "DK-MSZP-Párbeszéd (%)",
        "Momentum (%)",
        "MKKP (%)",
        "Mi Hazánk (%)",
        "Független (%)"  # Új oszlop
    ]

    total_population = df['Népesség'].sum()

    # EP országos eredmények számítása a fájlból
    ep_eredmenyek = {
        party: (df['Népesség'] * df[party] / 100).sum() / total_population * 100
        for party in parties
    }

    # Arányszám számítása: körzeti % / országos %
    aranyok = {
        party: df[party] / ep_eredmenyek[party]
        for party in parties
    }

    # Előrejelzett körzeti %-ok
    pred_szazalek_df = pd.DataFrame({
        party: aranyok[party] * orszagos_eredmenyek[party.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd")]
        for party in parties
    })

    # Korrigálás, ha több mint 100%
    row_sums = pred_szazalek_df.sum(axis=1)
    faktor = pd.Series(1.0, index=row_sums.index)
    overshoot = row_sums > 100
    faktor[overshoot] = 100.0 / row_sums[overshoot]
    pred_szazalek_df = pred_szazalek_df.mul(faktor, axis=0)

    # Körzeti szavazatszám
    korzeti_szavazok = (df['Népesség'] * reszvetel_szazalek / 100).round().astype(int)
    pred_szavazat_df = pd.DataFrame({
        party: (pred_szazalek_df[party] / 100 * korzeti_szavazok).round().astype(int)
        for party in parties
    })

    # Átszavazás alkalmazása körzetenként
    taktikai_atszavazas = taktikai_atszavazas or {}
    taktikai_atszavazas_korzet = taktikai_atszavazas_korzet or {}

    for idx, korzet in enumerate(df["Körzet"]):
        # Ellenőrizzük, van-e egyedi átszavazási szabály a körzetre
        if korzet in taktikai_atszavazas_korzet:
            atszavazas_szabalyok = taktikai_atszavazas_korzet[korzet]
        else:
            atszavazas_szabalyok = taktikai_atszavazas

        # Alkalmazzuk az átszavazási szabályokat
        for party, (arany, cel_party) in atszavazas_szabalyok.items():
            if party in pred_szavazat_df.columns and cel_party in pred_szavazat_df.columns:
                athelyezendo = int(round(pred_szavazat_df.at[idx, party] * arany))
                pred_szavazat_df.at[idx, party] -= athelyezendo
                pred_szavazat_df.at[idx, cel_party] += athelyezendo
            else:
                print(f"Figyelmeztetés: Érvénytelen párt vagy célpárt a körzetben: {korzet}, párt: {party}, cél: {cel_party}")

    # Új százalékok kiszámítása az áthelyezés után
    total_votes_per_district = pred_szavazat_df.sum(axis=1)
    pred_szazalek_df_adjusted = (pred_szavazat_df.div(total_votes_per_district, axis=0) * 100).round(2)

    # Egyéni győztesek, töredék, kompenzáció
    egyeni_gyoztesek = []
    egyeni_mand = {p: 0 for p in parties}
    toredek = {p: 0 for p in parties}
    komp = {p: 0 for p in parties}
    kulonbsegek = []

    for idx, row in pred_szavazat_df.iterrows():
        winner = row.idxmax()
        runnerup = row.drop(winner).max()
        kulonbseg_szavazat = row[winner] - runnerup
        total_votes = total_votes_per_district[idx]
        kulonbseg_szazalek = (kulonbseg_szavazat / total_votes * 100).round(2)

        egyeni_gyoztesek.append(winner.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd"))
        egyeni_mand[winner] += 1
        komp[winner] += int(kulonbseg_szavazat + 1)
        kulonbsegek.append(kulonbseg_szazalek)
        
        for p in parties:
            if p != winner:
                toredek[p] += int(row[p])

    # Lista szavazatok számítása
    ossz_szavazo = int(round(reszvetel_szazalek / 100 * total_population))
    listas = {
        p: int(round(orszagos_eredmenyek[p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd")] / 100 * ossz_szavazo)) + toredek[p] + komp[p] + (kulhoni_szavazatok.get(p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd"), 0) if kulhoni_szavazatok else 0)
        for p in parties
    }

    # 5% küszöb és D'Hondt
    jogosult = [
        p for p in parties
        if p != "Független (%)" and
        orszagos_eredmenyek[p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd")] >= 5.0
    ]
    list_mand = {p: 0 for p in jogosult}
    fix_mand = fix_mandatumok if fix_mandatumok else {}
    fix_db = sum(fix_mand.get(p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd"), 0) for p in parties)
    dhondt_helyek = 93 - fix_db

    for _ in range(dhondt_helyek):
        ertek = {p: listas[p] / (1 + list_mand[p]) for p in jogosult}
        nyertes = max(ertek, key=ertek.get)
        list_mand[nyertes] += 1

    for p in parties:
        nev = p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd")
        list_mand[p] = list_mand.get(p, 0) + fix_mand.get(nev, 0)

    osszes_mand = {p: egyeni_mand[p] + list_mand[p] for p in parties}

    # Kimenet
    print("\nMandátumösszesítő:")
    for p in parties:
        nev = p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd")
        print(f"{nev}: Összesen: {osszes_mand[p]}, Egyéni: {egyeni_mand[p]}, Listás: {list_mand[p]}, Szavazat: {listas[p]}")

    # Eredmény fájl
    out_df = pd.DataFrame({
        "Körzet": df["Körzet"],
        **{p.replace(" (%)", "").replace("Mi-Hazánk", "Mi Hazánk").replace("DK-MSZP-Párbeszéd", "DK-MSZP-Párbeszéd"): 
           pred_szazalek_df_adjusted[p].round(2) for p in parties},
        "Győztes": egyeni_gyoztesek,
        "Különbség": kulonbsegek
    })

    out_df.to_csv(output_path, sep=';', encoding='utf-8-sig', index=False)
    return egyeni_mand, list_mand, osszes_mand, out_df



In [32]:
# Futtatás bemeneti paraméterekkel
mandatumkalkulacio(
    csv_path="2024_ep_input_korzetek_bovitett.csv",
    orszagos_eredmenyek={
        "Fidesz": 40.0,
        "Tisza": 45.0,
        "DK-MSZP-Párbeszéd": 5.0,
        "Momentum": 3.0,
        "MKKP": 3.0,
        "Mi Hazánk": 5.0,
        "Független": 1.0  # Hozzáadott kulcs
    },
    reszvetel_szazalek=70.0,
    kulhoni_szavazatok={
        "Fidesz": 400000,
        "Tisza": 0,
        "DK-MSZP-Párbeszéd": 0,
        "Momentum": 0,
        "MKKP": 0,
        "Mi Hazánk": 0,
        "Független": 0  # Hozzáadott kulcs
    },
    fix_mandatumok={
        "Fidesz": 1,
        "Tisza": 0,
        "DK-MSZP-Párbeszéd": 0,
        "Momentum": 0,
        "MKKP": 0,
        "Mi Hazánk": 0,
        "Független": 0  # Hozzáadott kulcs
    },
    taktikai_atszavazas={
        #"Mi Hazánk (%)": (0.5, "Fidesz (%)"),
        #"DK-MSZP-Párbeszéd (%)": (0.5, "Tisza (%)")
    },
    taktikai_atszavazas_korzet={
        "Budapest 01": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 02": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 03": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 04": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 05": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 06": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 07": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 08": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 09": {
            "Tisza (%)": (0.7, "Független (%)")
        },
          "Budapest 10": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 11": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 12": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 13": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 14": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 15": {
            "Tisza (%)": (0.7, "Független (%)")
        },
        "Budapest 16": {
            "Tisza (%)": (0.7, "Független (%)")
        }
    },
    output_path="mandatum_kalkulacio_eredmeny.csv"
)


Mandátumösszesítő:
Fidesz: Összesen: 84, Egyéni: 40, Listás: 44, Szavazat: 3975911
Tisza: Összesen: 89, Egyéni: 50, Listás: 39, Szavazat: 3608307
DK-MSZP-Párbeszéd: Összesen: 5, Egyéni: 0, Listás: 5, Szavazat: 539462
Momentum: Összesen: 0, Egyéni: 0, Listás: 0, Szavazat: 322958
MKKP: Összesen: 0, Egyéni: 0, Listás: 0, Szavazat: 323137
Mi Hazánk: Összesen: 5, Egyéni: 0, Listás: 5, Szavazat: 540244
Független: Összesen: 16, Egyéni: 16, Listás: 0, Szavazat: 148402


({'Fidesz (%)': 40,
  'Tisza (%)': 50,
  'DK-MSZP-Párbeszéd (%)': 0,
  'Momentum (%)': 0,
  'MKKP (%)': 0,
  'Mi Hazánk (%)': 0,
  'Független (%)': 16},
 {'Fidesz (%)': 44,
  'Tisza (%)': 39,
  'DK-MSZP-Párbeszéd (%)': 5,
  'Mi Hazánk (%)': 5,
  'Momentum (%)': 0,
  'MKKP (%)': 0,
  'Független (%)': 0},
 {'Fidesz (%)': 84,
  'Tisza (%)': 89,
  'DK-MSZP-Párbeszéd (%)': 5,
  'Momentum (%)': 0,
  'MKKP (%)': 0,
  'Mi Hazánk (%)': 5,
  'Független (%)': 16},
              Körzet  Fidesz  Tisza  DK-MSZP-Párbeszéd  Momentum  MKKP  \
 0    Bács-Kiskun 01   41.62  44.09               3.59      1.62  2.47   
 1    Bács-Kiskun 02   35.14  50.11               3.49      2.36  3.19   
 2    Bács-Kiskun 03   44.79  40.77               4.31      1.66  1.69   
 3    Bács-Kiskun 04   45.87  40.18               3.05      1.65  2.51   
 4    Bács-Kiskun 05   43.99  37.56               4.37      1.69  2.57   
 ..              ...     ...    ...                ...       ...   ...   
 101     Veszprém 03   4

In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import unidecode
import numpy as np
from datetime import date
import os

# Színek pártok szerint
PART_COLORS = {
    "Fidesz": ((0xE4, 0xB8, 0x9B), (0xBB, 0x4D, 0x06)),
    "Tisza": ((0xA1, 0xB0, 0xD2), (0x24, 0x3B, 0x72)),
    "DK-MSZP-Párbeszéd": ((245, 163, 183), (92, 10, 30)),
    "Momentum": ((211, 165, 225), (58, 12, 72)),
    "MKKP": ((177, 225, 183), (24, 72, 30)),
    "Mi Hazánk": ((255, 243, 163), (102, 90, 10)),
    "Független": ((0xF5, 0xF5, 0xF5), (0x33, 0x33, 0x33))  # Legvilágosabb szürke -> sötétszürke
}

# Megjelenítési nevek a legendában
DISPLAY_NAMES = {
    "Fidesz": "Fidesz",
    "Tisza": "Tisza",
    "DK-MSZP-Párbeszéd": "DK-MSZP",
    "Momentum": "Momentum",
    "MKKP": "MKKP",
    "Mi Hazánk": "Mi Hazánk",
    "Független": "Független"
}

GREY = "#F5F5F5"  # Legvilágosabb szürke alapértelmezett színként

def korzet_to_svg_id(korzetnev):
    s = unidecode.unidecode(korzetnev.lower()).replace("–", "-").replace(" ", "-")
    helyettesites = {
        "borsod-abauj-zemplen": "borsod", "szabolcs-szatmar-bereg": "szabolcs",
        "jasz-nagykun-szolnok": "jasz", "gyor-moson-sopron": "gyor",
        "komarom-esztergom": "komarom", "bacs-kiskun": "bacs",
        "csongrad-csanad": "csongrad", "hajdu-bihar": "hajdu"
    }
    for regi, uj in helyettesites.items():
        s = s.replace(regi, uj)
    return s

def safe_max(series):
    val = series.max()
    return val if not np.isnan(val) else 0.0

def safe_min(series):
    val = series.min()
    return val if not np.isnan(val) else 0.0

def interpolate_color(c1, c2, t):
    r = int(c1[0] + (c2[0] - c1[0]) * t)
    g = int(c1[1] + (c2[1] - c1[1]) * t)
    b = int(c1[2] + (c2[2] - c1[2]) * t)
    return f"#{r:02x}{g:02x}{b:02x}"

def szin_kulonbseg_alapjan(row, shade_threshold):
    gyoztes = row["Győztes"]
    kulonbseg = abs(row["Különbség"])
    if gyoztes in PART_COLORS:
        light, dark = PART_COLORS[gyoztes]
        t = min(kulonbseg / shade_threshold, 1.0) if shade_threshold > 0 else 0.0
        return interpolate_color(light, dark, t)
    return GREY

def rajzolj_legendat(soup, kulonbseg_minmax, shade_threshold):
    defs = soup.new_tag("defs")

    def create_gradient(id_, c_light, c_dark, min_val, max_val):
        grad = soup.new_tag("linearGradient", id=id_, x1="0%", y1="0%", x2="100%", y2="0%")
        t_min = min_val / shade_threshold if shade_threshold > 0 else 0.0
        t_max = min(max_val / shade_threshold, 1.0)
        grad.append(soup.new_tag("stop", offset="0%", **{"stop-color": interpolate_color(c_light, c_dark, t_min)}))
        grad.append(soup.new_tag("stop", offset="100%", **{"stop-color": interpolate_color(c_light, c_dark, t_max)}))
        return grad

    active_parties = [(party, PART_COLORS[party], kulonbseg_minmax[party]) for party in PART_COLORS if kulonbseg_minmax[party][1] > 0]

    for party, (c_light, c_dark), (min_val, max_val) in active_parties:
        grad_id = f"grad_{unidecode.unidecode(party).replace(' ', '-')}"
        defs.append(create_gradient(grad_id, c_light, c_dark, min_val, max_val))

    soup.svg.insert(0, defs)
    group = soup.new_tag("g", id="legend", transform="translate(50, 30)")

    n = len(active_parties)
    scale_factor = 1.0
    layout = []

    if n == 1:
        layout = [(0, 0)]
    elif n == 2:
        layout = [(0, 0), (0, 1)]
    elif n == 3:
        scale_factor = 0.8
        layout = [(0, i) for i in range(3)]
    elif n == 4:
        scale_factor = 0.7
        layout = [(0, i) for i in range(2)] + [(1, i) for i in range(2)]
    elif n == 5:
        scale_factor = 0.7
        layout = [(0, i) for i in range(3)] + [(1, i) for i in range(2)]
    elif n == 6:
        scale_factor = 0.6
        layout = [(0, i) for i in range(3)] + [(1, i) for i in range(3)]
    elif n >= 7:
        scale_factor = 0.6
        layout = [(0, i) for i in range(3)] + [(1, i) for i in range(3)] + [(2, i) for i in range(n - 6)]

    box_width = 260 * scale_factor
    box_height = 30 * scale_factor
    font_size = int(18 * scale_factor)
    label_size = int(16 * scale_factor)
    spacing_x = 400 * scale_factor
    spacing_y = 80 * scale_factor

    for (col, row_index), (party, (c_light, c_dark), (min_val, max_val)) in zip(layout, active_parties):
        x = col * spacing_x
        y = row_index * spacing_y
        grad_id = f"grad_{unidecode.unidecode(party).replace(' ', '-')}"
        rect = soup.new_tag("rect", x=str(x), y=str(y), width=str(box_width), height=str(box_height), fill=f"url(#{grad_id})")
        group.append(rect)

        for pos, val in zip([0, box_width / 2, box_width], [min_val, (min_val + max_val)/2, max_val]):
            label = soup.new_tag("text", x=str(x + pos), y=str(y + box_height + 18), fill=interpolate_color(c_light, c_dark, 0.8),
                                 **{"font-size": str(label_size), "text-anchor": "middle", "font-weight": "bold"})
            label.string = f"{val:.2f}%"
            group.append(label)

        text = soup.new_tag("text", x=str(x + box_width + 12), y=str(y + box_height - 3), fill=interpolate_color(c_light, c_dark, 0.8),
                            **{"font-size": str(font_size), "font-weight": "bold"})
        text.string = DISPLAY_NAMES.get(party, party)
        group.append(text)

    soup.svg.append(group)

def nagyits_budapestet(soup):
    budapest_ids = [f"budapest-{i:02}" for i in range(1, 17)]
    scale = 5.5
    translate_x = 1000
    translate_y = 750
    rotate_deg = 7
    pivot_x = 244.1
    pivot_y = 172.8
    stroke_width = 0.05

    for korzet_id in budapest_ids:
        path = soup.find("path", {"id": korzet_id})
        if path:
            new_path = soup.new_tag("path", d=path["d"])
            new_path["id"] = korzet_id + "-zoom"
            szin = path.get("style", "").split(";")[0]
            new_path["style"] = f"{szin}; stroke: #000; stroke-width: {stroke_width};"
            new_path["transform"] = (
                f"translate({translate_x},{translate_y}) scale({scale}) rotate({rotate_deg}) translate({-pivot_x},{-pivot_y})"
            )
            soup.svg.append(new_path)

def gradient_terkep_svg(svg_path, korzet_df, shade_threshold=30, bp_zoom_enabled=True):
    today = date.today().isoformat()
    os.makedirs("gradient_terkep", exist_ok=True)
    suffix = "_teljes_bp" if bp_zoom_enabled else "_teljes"
    output_path = f"gradient_terkep/{today}{suffix}.svg"

    # Különbség számítás beépítése
    if "Különbség" not in korzet_df.columns:
        korzet_df["Különbség"] = 0.0
    else:
        korzet_df["Különbség"] = korzet_df["Különbség"].astype(float)  # Biztosítjuk, hogy float legyen
    required_columns = ["Körzet", "Győztes"] + list(PART_COLORS.keys())
    if not all(col in korzet_df.columns for col in required_columns):
        raise ValueError(f"A korzet_df-nek tartalmaznia kell a következő oszlopokat: {required_columns}")
    
    for i, row in korzet_df.iterrows():
        gy = row["Győztes"]
        if gy in PART_COLORS:
            max_ellenfel = max([row[p] for p in PART_COLORS if p != gy])
            korzet_df.at[i, "Különbség"] = row[gy] - max_ellenfel

    with open(svg_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "xml")

    svg_tag = soup.find("svg")
    viewbox = svg_tag.get("viewBox")
    if viewbox:
        x, y, w, h = map(float, viewbox.split())
        svg_tag["viewBox"] = f"{x} {y} {w * 2} {h * 2}"
        svg_tag["width"] = str(w * 2)
        svg_tag["height"] = str(h * 2)

    for path in soup.find_all("path"):
        path["transform"] = f"scale(2) {path.get('transform', '')}".strip()

    for text in soup.find_all("text"):
        text["font-size"] = str(float(text.get("font-size", 10)) * 2)
        if "x" in text.attrs:
            text["x"] = str(float(text["x"]) * 2)
        if "y" in text.attrs:
            text["y"] = str(float(text["y"]) * 2)

    kulonbseg_minmax = {}
    for party in PART_COLORS:
        part_vals = korzet_df[korzet_df["Győztes"] == party]["Különbség"].abs()
        kulonbseg_minmax[party] = (safe_min(part_vals), safe_max(part_vals))

    for _, row in korzet_df.iterrows():
        korzet_id = korzet_to_svg_id(row["Körzet"])
        path = soup.find("path", {"id": korzet_id})
        if path:
            szin = szin_kulonbseg_alapjan(row, shade_threshold)
            path["style"] = f"fill: {szin}; stroke: #000; stroke-width: 0.1;"

    if bp_zoom_enabled:
        print(">> Budapest nagyítása bekapcsolva.")
        nagyits_budapestet(soup)
    else:
        print(">> Budapest nagyítása kikapcsolva.")

    rajzolj_legendat(soup, kulonbseg_minmax, shade_threshold)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(str(soup))

    print(f">> SVG fájl elmentve: {output_path}")

# === Használat ===
korzet_df = pd.read_csv("mandatum_kalkulacio_eredmeny.csv", sep=";", encoding="utf-8-sig")
gradient_terkep_svg("2026_korzetek_alap.svg", korzet_df, shade_threshold=30, bp_zoom_enabled=True)

>> Budapest nagyítása bekapcsolva.
>> SVG fájl elmentve: gradient_terkep/2025-05-04_teljes_bp.svg
